#### Importing Libraries:

In [2]:
import os, sys
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random
import pickle

#### Defining Targets:

In [3]:
base_dir = "./ranzcr_clip_catheter_line_classification/"
df = pd.read_csv(os.path.join(base_dir, "train.csv"))
# df = df.drop(['StudyInstanceUID', 'PatientID'], axis=1)
df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8


#### Tackle data consistency:

In [4]:
#The data at hand includes:

my_df = pd.read_csv(os.path.join(base_dir, "my_data.csv"), header = None)
my_df

,0
0,1.2.826.0.1.3680043.8.498.11884337472077427136...
1,1.2.826.0.1.3680043.8.498.11884664327132209283...
2,1.2.826.0.1.3680043.8.498.11884987406928532272...
3,1.2.826.0.1.3680043.8.498.11885403975706303589...
4,1.2.826.0.1.3680043.8.498.11886085574279783392...
...,...
495,1.2.826.0.1.3680043.8.498.12082773887313156676...
496,1.2.826.0.1.3680043.8.498.12083551386327135012...
497,1.2.826.0.1.3680043.8.498.12083638663506990841...
498,1.2.826.0.1.3680043.8.498.12083662055639223919...


In [5]:
#deleting the extention ".jpg" from "my_data":

for i in range(0, len(my_df)):
    my_df[0][i] = my_df[0][i][:-4]

In [6]:
#finding the matched data:

temp_arr = df['StudyInstanceUID'].values.tolist()
lst_index = []

for i in range(0, len(my_df)):
    lst_index.append(temp_arr.index(my_df[0][i]))
    
# #A slower method of O(n^2):    
# for i in range(0, len(my_df)): #or using .find() method
#     for j in range(0, len(df)):
#         if my_df[0][0] == df['StudyInstanceUID'][j]:
#             lst_index.append(j)    
    
lst_index.sort()
# lst_index

In [7]:
#either deletes the excess data from the original dataset, or sorts them.

labels = df.iloc[lst_index, :]
labels

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
14,1.2.826.0.1.3680043.8.498.11974864693507588499...,0,0,0,0,0,0,0,0,0,1,0,b88984341
30,1.2.826.0.1.3680043.8.498.11909730476134310240...,0,0,0,0,0,0,0,0,0,1,0,e8460b753
242,1.2.826.0.1.3680043.8.498.12007322719604850756...,0,0,1,0,0,0,1,0,0,1,0,faf15123e
248,1.2.826.0.1.3680043.8.498.11889424946605247098...,0,0,0,0,0,0,0,0,1,1,0,1eb6ed982
262,1.2.826.0.1.3680043.8.498.12011647071471951833...,0,0,1,0,0,0,1,0,1,1,1,a2db76c34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29680,1.2.826.0.1.3680043.8.498.11931339765295264282...,0,0,1,0,0,0,1,0,1,1,1,8d26758a4
29805,1.2.826.0.1.3680043.8.498.11898758777651667391...,0,0,0,0,0,0,0,0,0,1,0,1f7fa4e6d
29888,1.2.826.0.1.3680043.8.498.11910678980456927176...,0,0,0,0,0,0,0,0,1,0,0,dfb9e08b4
29951,1.2.826.0.1.3680043.8.498.12044569043485413614...,0,0,0,0,0,0,0,0,0,1,0,a48997f66


In [8]:
#solving data leackage issue:

temp_arr = labels['PatientID'].values.tolist()
temp_lst = []
lst_id = []

for i in range(0, len(labels)):
    for j in range(i+1, len(labels)):
        if temp_arr[i] == temp_arr[j]:
            temp_lst.append(j) #labels.index[j]
    if temp_lst != []:
        temp_lst.insert(0, i) #labels.index[i]
        lst_id.append(temp_lst)
        temp_lst = []

# len(lst_id) #verified!
# lst_id
arr_id = np.array(lst_id)
arr_id

array([list([0, 488]), list([2, 204, 399]), list([11, 16, 408]),
       list([16, 408]), list([17, 333]), list([21, 63, 121, 281]),
       list([24, 320, 337, 373]), list([25, 125]), list([26, 271]),
       list([29, 83]), list([31, 109]), list([32, 213]),
       list([39, 117, 415, 459]), list([43, 103, 355]), list([49, 365]),
       list([51, 161, 471]), list([53, 398]), list([59, 76]),
       list([61, 100]), list([63, 121, 281]), list([65, 209]),
       list([74, 79, 321]), list([75, 492]), list([79, 321]),
       list([82, 137, 360]), list([96, 482]), list([98, 242, 299]),
       list([103, 355]), list([112, 227]), list([114, 116]),
       list([115, 499]), list([117, 415, 459]), list([121, 281]),
       list([133, 234]), list([137, 360]), list([140, 294]),
       list([145, 268]), list([149, 180, 489]), list([151, 168]),
       list([157, 158]), list([160, 372]), list([161, 471]),
       list([167, 401]), list([169, 430]), list([174, 348]),
       list([176, 226]), list([180, 489

In [9]:
#dropping unnecessary columns in the df "labels":

labels = labels.drop(['StudyInstanceUID', 'PatientID'], axis=1)
# labels

In [10]:
arr_labels = labels.values
arr_labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [11]:
arr_labels.shape

(500, 11)

#### Resizing images:

In [12]:
def load_resize_training_data(imageSize = 224, numSamples = 10000): #all image sizes: 650*650 #also changed from 10000 to 1000 due to my system memory.
    path = os.path.join(base_dir, "train/")
    trainingData = []
    i=0
    for img in os.listdir(path):
        if i == numSamples:
            break
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) #plt.imshow will not result in a very fair respresentation.
        resized_array = cv2.resize(img_array, (imageSize, imageSize))
        trainingData.append(resized_array) #train_df.at[i, 'combined_label']]
        i += 1

#     plt.imshow(trainingData[0], cmap = "gray")

    return trainingData

data_train = load_resize_training_data()
data_train

[array([[ 1,  2,  2, ...,  3,  2,  2],
        [ 3,  2,  2, ...,  4,  3,  1],
        [ 2,  2,  2, ...,  4,  3,  3],
        ...,
        [ 2,  3,  3, ..., 58, 38, 20],
        [ 2,  3,  3, ..., 61, 42, 24],
        [ 2,  2,  2, ..., 60, 44, 25]], dtype=uint8),
 array([[  9,   7,   7, ...,  19,  20,  20],
        [  7,   7,   7, ...,  18,  17,  20],
        [  7,   7,   7, ...,  17,  16,  18],
        ...,
        [ 93,  88,  85, ..., 165, 167, 176],
        [ 90,  88,  85, ..., 168, 169, 169],
        [ 85,  83,  79, ..., 173, 169, 172]], dtype=uint8),
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [247, 247, 245, ...,  28,  25,  30],
        ...,
        [103,  99, 132, ...,  90,  36,   6],
        [109, 128, 113, ...,  90,   4,   9],
        [113, 125,  67, ...,  70,  13,  11]], dtype=uint8),
 array([[  2,   2,   2, ...,  16,  17,  19],
        [  2,   3,   3, ...,  15,  15,  18],
        [  3,   3,   3, ...,  13,  14,  14],
       

In [13]:
len(data_train) #verified!

500

In [14]:
data_train[0].shape #verified!

(224, 224)

In [15]:
arr_train = np.array(data_train)

In [16]:
arr_train.shape

(500, 224, 224)

In [17]:
#So far, we have the x-ray images in the "arr_train" variable. -> saving to "train.pickle"
#And also the 11 labels in the "arr_labels" variable. -> saving to "target.pickle"
#And all the repeted PatientIDs in the "arr_id" variable. -> saving to "id.pickle"

#### Saving the files:

In [18]:
#save x data
pickle_out = open("train.pickle", "wb")
pickle.dump(arr_train, pickle_out)
pickle_out.close()
#save y data
pickle_out = open("target.pickle", "wb")
pickle.dump(arr_labels, pickle_out)
pickle_out.close()
#save PatientIds:
pickle_out = open("id.pickle", "wb")
pickle.dump(arr_id, pickle_out)
pickle_out.close()